In [1]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from scripts.gridsearch import text_grid

0.17


In [2]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [3]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [4]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


Create a model for the following grid search

In [5]:
# LR
clf_behavior_LR = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
clf_behavior_LR.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [6]:
clf_behavior_LR.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Now we are trying to implement grid search.

In [7]:
param_grid = {
    'clf__C': uniform(0.0001, 1000),
    'clf__penalty': ['l2', "l1"],
#     'clf__penalty': ['l2'],
    'clf__tol': uniform(0.0001, 0.001),
#     'clf__verbose': [0]
#     'features__text__tfidf__analyzer': ['char'],
#     'features__text__tfidf__ngram_range': [(2,5)]
}

param_grid.update(text_grid)

cv_kwargs = dict(
    n_iter=5,
    scoring=None,
    fit_params=None,
    n_jobs=-1,
    iid=True,
    refit=True,
    cv=3,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=0
)

In [ ]:
clf_behavior_LR_grid = RandomizedSearchCV(clf_behavior_LR, param_grid, **cv_kwargs)
clf_behavior_LR_grid.fit(X_train, y_train)

In [ ]:
# get the best estimator 
best_estimator_LR = clf_behavior_LR_grid.best_estimator_
print('best estimator: ', best_estimator_LR)
print('-------------------------- mean accuracy on the cross validation of best estimator --------------------------')
print('best score: ', clf_behavior_LR_grid.best_score_)
print('best score related parameters: ', clf_behavior_LR_grid.best_params_)

Test the best estimator performance on testing dataset

In [ ]:
# predict in testing set.
y_behavior_LR_grid = best_estimator_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_LR_grid, y_test)
f1_score_LR_grid = computeF1Score(y_test, y_behavior_LR_grid, average = None)
print('F1 Score: ', f1_score_LR_grid)
auc_score_LR_grid = computeAUCBehavior(best_estimator_LR, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('LR classifier AUC: ', auc_score_LR_grid)